# Modeling Weather Geographies using Scikit-Learn on Hadoop Data

In this notebook, we are using aggregated `TMAX` data by global weather stations in order to create a machine learning model to predict the daily maximum temperatures at any given latitude and longitude. Our model will take 3 continuous predictors: latitude, longitude, and elevation, and provide an estimated `TMAX` for a given day of the year.

Our input data will reside in HDFS for a registered Hadoop Integration system. To avoid having to copy the data from Hadoop into Watson Studio, we will use a remote Livy session to build the model _within Hadoop itself_. Then we will "pull" the model into Watson Studio and save it to your Watson Studio filesystem, making it available for use with other Watson Studio model management features.

<div class="alert alert-block alert-info">Note: The Scikit-Learn APIs that we use in this sample are single-threaded; this sample does *not* use Hadoop distributed computing to create the model. The reason we're using Hadoop in this example is purely for data purposes: we want to build the model <i>where the data resides</i>, instead of having to pull the data into Watson Studio.</div>

## Table of Contents
This notebook contains these main sections:

1. [Prerequisites](#Prerequisites)
2. [Create a Remote Livy Session](#Create_Livy_Session)
3. [The Data](#The_Data)
4. [The Model](#The_Model)
5. [Copy Models to Watson Studio Local](#Model_Copy_to_WSL)
6. [Save Models to Watson Studio Filesystem](#Save_Models_to_WSL_Filesystem)
7. [Cleanup the_Remote Livy Session](#Cleanup_Remote_Livy_Session)
8. [Summary](#Summary)

<a id='Prerequisites'></a>
## Prerequisites

### I) Create a custom image with `tqdm`
The model creation logic for this sample requires the python `tqdm` library. Since we will be running the model creation logic in a _remote_ Livy session, we will need to create a custom image which includes `tqdm`, and then configure our Livy session to use that image. In order to do this you can take the following steps:

#### A. Start an environment
From your project home page, use the `Environments` tab to _start_ a "`Jupyter with Python 2.7, ...`" environment.

#### B. Install `tqdm` into the environment
From your project home page, use the `Environments` tab to _launch a terminal_ shell for the environment that you started in Step A. When you are inside the terminal, type the following command to install `tqdm`:

```
conda install tqdm -y
```

When the command completes, you can `exit` the terminal.

#### C. Save the environment as a custom image
From your project home page, use the `Environments` tab to _save_ the environment that you edited in Step B.

### II) Register a Hadoop Integration system (Admin)

Ask your Watson Studio admin to use the **Admin Console => Hadoop Integration** option to:

  * Register a Hadoop Integration system.

    ** NOTE: Installation and configuration of IBM's Hadoop Integration (`HI`) service on a Hadoop cluster must be done by a Hadoop admin _before_ that system can be registered with your Watson Studio. **


  * Push the custom image you created in Step I above to the registered HI system.
  
When your admin indicates that your custom image has been pushed to the registered HI system, you can proceed with this sample notebook.

In [1]:
# Imports needed for the cells which run locally on Watson Studio.
import dsx_core_utils
import pandas as pd
from sklearn.externals import joblib

<a id='Create_Livy_Session'></a>
## Create a Remote Livy Session

First, let's get a list of registered Hadoop Integration systems. Look for a system that has the `imageId` of your custom image.

In [ ]:
DSXHI_SYSTEMS = dsx_core_utils.get_dsxhi_info(showSummary=True)

Configure the spark session that we will use when running on the selected registered HI system. In this case we want the session to start with 2G memory and we only need a single executor since we're using single-threaded Scikit-Learn APIs. **NOTE**: `myConfig` here is optional; if you prefer to use default configs you can omit this cell and remove the `addlConfig` argument in the next cell.

In [3]:
myConfig={
 "queue": "default",
 "driverMemory": "2G",
 "numExecutors": 1
}

In [4]:
# Set up sparkmagic to connect to the selected registered HI
# system with the specified configs. **NOTE** This notebook
# requires Spark 2, so you should set 'livy' to 'livyspark2'.
HI_CONFIG = dsx_core_utils.setup_livy_sparkmagic(
  system="Zinc", 
  livy="livyspark2",
  imageId="py27-tqdm",
  addlConfig=myConfig)

# (Re-)load spark magic to apply the new configs.
%reload_ext sparkmagic.magics

sparkmagic has been configured to use https://zinc1.fyre.ibm.com:8443/gateway/dsx-loc-chell-375-master-1/livy2/v1 with image Jupyter with Python 2.7 + TQDM
success configuring sparkmagic livy.


Now, let's capture some state about the configured Hadoop Integraton system, to be used later in this notebook. Then start up a new, remote Livy session to connect to that HI system. **NOTE**: Depending on a) the resources available in the remote Hadoop system and b) the speed of your cluster, attempts to start the session might report errors due to timeout or due to a session coming up `dead`.  In such cases you should run **`%spark cleanup`** as a separate cell, then re-run this cell again.  If session creation continues to fail, contact the Hadoop admin of the target Hadoop cluster to see if everything is configured as expected.

In [5]:
session_name = 'sksess1'
livy_endpoint = HI_CONFIG['LIVY']
webhdfs_endpoint = HI_CONFIG['WEBHDFS']
%spark add -s $session_name -l python -k -u $livy_endpoint

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
230,application_1533478912530_1339,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


For reference / debugging: Print out the name of the Hadoop node to which the remote session has been assigned. When "local" files are created within the remote session, they will be written to this node. All of the Yarn container artifacts (workspace and temp files) will exist on this node as well.

In [6]:
%%spark -s $session_name
import socket
print("Remote Livy session driver: {}".format(socket.gethostname()))

Remote Livy session driver: ales3.fyre.ibm.com

The following cell, and all subsequent cells which have **`%%spark`** as their first line, will run *remotely*, i.e. within a Yarn container that exists on the registered Hadoop Integration system.

In [7]:
%%spark -s $session_name

# Declare imports needed for all of the cells that will run remotely.

import numpy as np
import pandas as pd
import getpass, time, os

from pyspark import SparkFiles
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from subprocess import Popen, PIPE, STDOUT
from tqdm import tqdm

# Load IBM Hadoop Integration utilities to facilitate remote functionality.
hi_utils_lib = os.getenv("HI_UTILS_PATH", "")
sc.addPyFile("hdfs://{}".format(hi_utils_lib))
import hi_core_utils

# Declare some target directory paths that will be used when dealing with data and model artifacts.
input_csv_name = "seasonal_data.csv"
hdfs_dataset_dir = "/user/{}/datasets".format(getpass.getuser())
input_ds = "{}/{}".format(hdfs_dataset_dir, input_csv_name)

<a id='The_Data'></a>
## The Data
Load our test data into HDFS. For purposes of this sample our data is small and comes from a remote URL. We do not _need_ to put it into HDFS--but we choose to do so for demonstration purposes. In a real scenario the desired data should already be loaded into HDFS.

In [8]:
%%spark -s $session_name

# Download the data file to the local fs of the Hadoop node that is "driving"
# our active livy session.
sc.addFile("https://raw.githubusercontent.com/IBMDataScience/DSX-DemoCenter/master/weatherGeographies/data_assets/seasonal_data.csv")

# Create a target directory on HDFS.
hi_core_utils.run_command("hdfs dfs -mkdir -p {}".format(hdfs_dataset_dir))

# Upload the saved csv file to HDFS.
hi_core_utils.run_command("hdfs dfs -put -f {}/{} {}".format(
    SparkFiles.getRootDirectory(), input_csv_name, hdfs_dataset_dir))

Now use spark to read the data, as a **spark data frame**, from HDFS.

In [9]:
%%spark -s $session_name
df_data_1 = spark.read.format(
    "org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").option(
    "header", "true").option("inferSchema", "true").load(input_ds)

<a id='The_Model'></a>
## The Model
Let's first split the data by columns into features and response variables. **`sdf_x`** and **`sdf_y`** will hold **spark dataframes** at this point.

In [10]:
%%spark -s $session_name
sdf_x = df_data_1[['elevation','latitude','longitude']]
sdf_y = df_data_1[['21-Mar','21-Jun','21-Sep','21-Dec']]

Convert the spark data frames to pandas since the Scikit-Learn APIs that we use have to work with pandas (and therefore they are _not_ distributed).

In [11]:
%%spark -s $session_name
x = sdf_x.toPandas()
y = sdf_y.toPandas()

Split the data further into training and testing sets. We will fit a **Gradient Boosting Regressor** model. We should first tune our model for the hyperparameter `n_estimators` to discover what is the maximum number of estimators we should use.

In [12]:
%%spark -s $session_name
x_init, x_test, y_init, y_test = train_test_split(x, y['21-Jun'], test_size=.25)
x_train, x_val, y_train, y_val = train_test_split(x_init, y_init, test_size=.25)

Now we can iterate over a specified range to determine the best value for n_estimators. **Note**: Since we're running inside of a _remote_ Livy session, UI-based functionality will not work. Thus you will *not* see the progress indicator bar that you might expect to see if you were running this code locally...

In [ ]:
%%spark -s $session_name
n_est = list(range(1,201))
val_mad = list()
for n in tqdm(n_est):
    val_model = GradientBoostingRegressor(n_estimators=n)
    val_model.fit(x_train,y_train)
    val_pred = val_model.predict(x_val)
    val_mad.append(mean_absolute_error(y_val,val_pred))

### Finding the Elbow
In order to find the optimal value of `n_estimators`, we'll calculate the equation of the line between our minimum and maximum value of `n_estimators`, and find out the value of `n` in which the validation error is the furthest away from the line.

In [14]:
%%spark -s $session_name
slope = (val_mad[-1]-val_mad[0])/(n_est[-1]-n_est[0])
intercept = val_mad[0] - slope*n_est[0]
best_n = (np.array(n_est)*slope+intercept-np.array(val_mad)).argmax()
print("The best value of n is {}.".format(n_est[best_n]))

The best value of n is 33.

Using the best number of estimators from above, we'll fit the model we'll use for prediction on the test set.

In [15]:
%%spark -s $session_name
gbtr = GradientBoostingRegressor(n_estimators=n_est[best_n])
gbtr.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=33, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [16]:
%%spark -s $session_name
y_pred = gbtr.predict(x_test)
print("Mean Absolute Error: {}\nR^2 value: {}".format(mean_absolute_error(y_pred,y_test),gbtr.score(x_test,y_test)))

Mean Absolute Error: 2.0153655727
R^2 value: 0.782325989428

Remember that we are dealing with degrees Celsius. Our mean error in this case is around 2 degrees. We also have a strong $R^2$ value.

Now let's fit the models on the entirety of the data.

In [17]:
%%spark -s $session_name
gbtr.fit(x,y['21-Jun'])
gbtr_dec = GradientBoostingRegressor(n_estimators=n_est[best_n])
gbtr_dec.fit(x,y['21-Dec'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=33, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

<a id='Model_Copy_to_WSL'></a>
## Copy Models to Watson Studio Local

The models now exist within the memory of the remote livy session. In order to use them in Watson Studio model management, we need to copy them to the local Watson Studio environment.  This is done in two parts.

### Write Models to HDFS
First, in the _remote_ session, we use a Hadoop Integration utility method to write the models to HDFS, along with some associated metadata.

In [18]:
%%spark -s $session_name
print(hi_core_utils.write_model_to_hdfs(model=gbtr, model_name="gbtr_jun"))
print(hi_core_utils.write_model_to_hdfs(model=gbtr_dec, model_name="gbtr_dec"))

{'path': 'hdfs:///user/user1/.dsxhi/models/gbtr_jun/2/model', 'version': 2, 'name': 'gbtr_jun', 'latest_version': 2}
{'path': 'hdfs:///user/user1/.dsxhi/models/gbtr_dec/2/model', 'version': 2, 'name': 'gbtr_dec', 'latest_version': 2}
/hadoop/yarn/local/usercache/user1/appcache/application_1533478912530_1339/container_e32_1533478912530_1339_01_000001/py27-tqdm.tar.gz/conda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.

### Load Models from HDFS into Watson Studio Local
Then, on the Watson Studio _local_ side, we use a Watson Studio Local utility method to load the model from HDFS into memory. Note that the model names we use here should match the ones we used in the previous cell, when we wrote the models to HDFS.

Note also that this cell **does not** begin with the **`%%spark`** line, which means it is running locally in your Watson Studio.

In [19]:
loc_gbtr_jun = dsx_core_utils.load_model_from_hdfs(webhdfs_endpoint, model_name="gbtr_jun")
loc_gbtr_dec = dsx_core_utils.load_model_from_hdfs(webhdfs_endpoint, model_name="gbtr_dec")

Model loaded from hdfs:///user/user1/.dsxhi/models/gbtr_jun/2/model
Model loaded from hdfs:///user/user1/.dsxhi/models/gbtr_dec/2/model


<a id='Save_Models_to_WSL_Filesystem'></a>
## Save Models to Watson Studio Filesystem
We can now save `scikit-learn` models to the Watson Studio filesystem for publishing, scoring, deployment, and evaluations.

When invoking the `save` function we want to pass pandas dataframes for **`x_test`** and **`y_test`** as arguments. By doing so we allow the `save` function to a) determine the schema of the test data automatically, and b) find an example row that can be used elsewhere in the WSL model management UI (ex. for real-time scoring).

At this point the desired dataframes exist within the _remote_ Livy session, which means they are not directly accessible from the local notebook session. However, we can use `sparkmagic` to pull a **single** row ("`-n 1`") from each of those views.  This allows us to get the minimum necessary information we need from the test data **without** having to read the full datasets from HDFS.

Here we load one row of data from each dataframe into **`x`** and **`y`**, respectively.

In [20]:
%%spark -s $session_name -n 1 -o x
x = sdf_x

In [21]:
%%spark -s $session_name -n 1 -o y
y = sdf_y

Now that we have our **`x_test`** and **`y_test`** dataframes, let's import the `save` function from the `dsx_ml.ml` library. The save function takes a few additional arguments which are listed below.

In [22]:
from dsx_ml.ml import save

Now we can save both the June 21 and December 21 models.

**NOTE**: Since we're using dataframes with a **single** row, i.e. partial data, we choose to skip calculation of performance metrics for the saved model ("`skip_metrics = True`") since metrics based on a single row are not useful.

In [23]:
save(model = loc_gbtr_jun,
     name = 'Jun21 Scikit via Hadoop',
     x_test = x,
     y_test = pd.DataFrame(y['21-Jun']),
     algorithm_type = 'Regression',
     skip_metrics = True)

Using TensorFlow backend.


{'path': '/user-home/1001/DSX_Projects/Models on Hadoop/models/Jun21 Scikit via Hadoop/17',
 'scoring_endpoint': 'https://dsxl-api/v3/project/score/Python27/scikit-learn-0.19/Models%20on%20Hadoop/Jun21%20Scikit%20via%20Hadoop/17'}

In [24]:
save(model = loc_gbtr_dec,
     name = 'Dec21 Scikit via Hadoop',
     x_test = x,
     y_test = pd.DataFrame(y['21-Dec']),
     algorithm_type = 'Regression',
     skip_metrics = True)

{'path': '/user-home/1001/DSX_Projects/Models on Hadoop/models/Dec21 Scikit via Hadoop/17',
 'scoring_endpoint': 'https://dsxl-api/v3/project/score/Python27/scikit-learn-0.19/Models%20on%20Hadoop/Dec21%20Scikit%20via%20Hadoop/17'}

<a id='Cleanup_Remote_Livy_Session'></a>
## Cleanup the Remote Livy Session
We're done with our models and we have successfully saved them to Watson Studio. Let's cleanup our remote Livy session. 
This will terminate the session and release resources back to the remote Hadoop Integration system.

In [25]:
%spark cleanup

<a id='Summary'></a>
## Summary
In this notebook you learned how to create a `scikit-learn` model _on a registered Hadoop Integration system_, allowing you to create the model where the data resides, instead of having to copy your data into the Watson Studio environment.  Once the model was created you were able to save it in the Watson Studio environment, where it can now be used as input for other Watson Studio model management features.

<div class="alert alert-block alert-info">Note: To save resources and get the best performance please use the code below to stop the kernel before exiting your notebook.</div>

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

<hr>
Copyright &copy; IBM Corp. 2018. Released as licensed Sample Materials.